In [1]:
# ********************** REFERENCES *****************************

"""
reading from a CSV file
- https://www.youtube.com/watch?v=eEIr70i8vbs

writing to a CSV file
- https://www.youtube.com/watch?v=hmYdzvmcTD8
- https://stackoverflow.com/questions/34864695/saving-prediction-results-to-csv

adding a new column
- https://discuss.codecademy.com/t/can-we-add-a-new-column-at-a-specific-position-in-a-pandas-dataframe/355842
"""

# ***************************************************************

'\nreading from a CSV file\n- https://www.youtube.com/watch?v=eEIr70i8vbs\n\nwriting to a CSV file\n- https://www.youtube.com/watch?v=hmYdzvmcTD8\n- https://stackoverflow.com/questions/34864695/saving-prediction-results-to-csv\n\nadding a new column\n- https://discuss.codecademy.com/t/can-we-add-a-new-column-at-a-specific-position-in-a-pandas-dataframe/355842\n'

In [2]:
# ************************ IMPORTS ******************************

# -------------- Modelling Packages --------------
# For modeling
from keras.models import Model
from keras.layers import Concatenate, Input, Dense, Embedding
from keras.layers import LSTM, Bidirectional, SpatialDropout1D
from keras.layers import TimeDistributed
from keras.models import load_model
import tensorflow as tf

# Callback Functions
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

# For Timestamping Models
import time

# -------------- Preprocessing Packages --------------
# For tokenizing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# -------------- General Packages --------------
# General Use
import pandas as pd
import numpy as np

# For Saving Files
import pickle
import os

# ***************************************************************

Using TensorFlow backend.
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: Future

In [3]:
# ******************* LOADING THE DATASET ***********************

# Given the split dataset directory, return the train/test split
def load_dataset(split_data_dir):
    pickle_in = open(split_data_dir+'X_train.pickle','rb')
    X_train = pickle.load(pickle_in)
    
    pickle_in = open(split_data_dir+'X_test.pickle','rb')
    X_test = pickle.load(pickle_in)

    pickle_in = open(split_data_dir+'y_train.pickle','rb')
    y_train = pickle.load(pickle_in)

    pickle_in = open(split_data_dir+'y_test.pickle','rb')
    y_test = pickle.load(pickle_in)
    return X_train,X_test,y_train,y_test

def load_tokenizer(tokenizer_dir):
    pickle_in = open(tokenizer_dir,'rb')
    t = pickle.load(pickle_in)
    return t
    
split_data_dir = './split_data/'
X_train,X_test,y_train,y_test = load_dataset(split_data_dir)

tokenizer_dir = 'tokenizer.pickle'
t = load_tokenizer(tokenizer_dir)

# ***************************************************************

In [4]:
# ****************** PREPROCESSING FUNCTION *********************

# Filters corrupted, unusable/unused data from the dataset
def filter_dataset(df):
    df = df.drop(columns=['id','tid1','tid2']) # drop id columns
    df = df.drop(columns=['title1_zh','title2_zh']) # drop chinese columns

    # Remove symbols
    df['title1_en'] = df['title1_en'].str.replace('[^a-zA-Z0-9 ]','')
    df['title2_en'] = df['title2_en'].str.replace('[^a-zA-Z0-9 ]','')

    # Replace empty strings with NaN values
    df['title1_en'].replace('', np.nan, inplace=True)
    df['title2_en'].replace('', np.nan, inplace=True)
    
    # Remove rows with no label
    labels = ['unrelated','agreed','disagreed']
    df = df[df.label.isin(labels)]
    
    # Remove Unnamed columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Drop rows with null values
    df = df.dropna()
    return df

# Returns a datset with an equal sampling of each label
def equalize_dataset_labels(df, seed=1):
    # Get minimum label count
    n = df.label.value_counts().min()
    
    # Grabbing equal amounts of training data from each class
    dfa = df[df['label']=='unrelated'].sample(n,random_state=seed)
    dfb = df[df['label']=='agreed'].sample(n,random_state=seed)
    dfc = df[df['label']=='disagreed'].sample(n,random_state=seed)
    
    # Recombine dataset and shuffle
    df = pd.concat([dfa,dfb,dfc])
    df = df.sample(frac=1,random_state=seed)
    return df

# Convert labels to integers for predictions
def encode_labels(df):
    # encoding the labels
    labels = {'unrelated':0,'agreed':1,'disagreed':2}
    df['label'].replace(labels,inplace=True)
    df = df.reset_index()
    return df
# ***************************************************************

In [5]:
# ******************* TOKENIZING FUNCTIONS **********************

# Create a word tokenizer given dataframe(s)
def create_tokenizer(*data, num_words=None, lower=True, split=' ', oov_token=None, filename='tokenizer'):
    # create the tokenizer
    t = Tokenizer(num_words=num_words, lower=lower, split=split, oov_token=oov_token)
    
    # fit tokenizer
    for df in data:
        t.fit_on_texts(df['title1_en'])  
        t.fit_on_texts(df['title2_en'])  
    
    # save for future use
    pickle_out = open(filename+'.pickle', 'wb')
    pickle.dump(t,pickle_out)
    pickle_out.close()
    return t

# Tokenizes titles and encodes labels, trains a word tokenizer that is saved to a file
def tokenize(t, df, maxlen=20):
    # fit the tokenizer on the documents  
    data1 = pad_sequences(sequences=t.texts_to_sequences(df['title1_en']), maxlen=maxlen)
    data2 = pad_sequences(sequences=t.texts_to_sequences(df['title2_en']), maxlen=maxlen)
    
    # recombine
    df = pd.DataFrame(np.concatenate((data1,data2),axis=1)).join(df['label'])
    return df

# ***************************************************************

In [6]:
embeddings_index = dict()

EMBED_SIZE = 100
LSTM_SIZE = 2

# -------------- Compile Parameters --------------
activation = 'softmax'
optimizer = 'RMSProp'
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']

SENTENCE_SIZE = int(X_train.shape[1]/2)
vocab_size = len(t.word_index) + 1

"""

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, EMBED_SIZE))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# FIRST MODEL: TITLE1_EN
first_input = Input(shape=(SENTENCE_SIZE,))
m1 = Embedding(vocab_size,
                EMBED_SIZE,
                weights=[embedding_matrix],
                input_length=SENTENCE_SIZE,
                trainable=False)(first_input)
m1 = Bidirectional(LSTM(LSTM_SIZE,dropout=0.2, recurrent_dropout=0.2))(m1)

# SECOND MODEL: TITLE2_EN
second_input = Input(shape=(SENTENCE_SIZE,))
m2 = Embedding(vocab_size,
                 EMBED_SIZE,
                 weights=[embedding_matrix],
                 input_length=SENTENCE_SIZE,
                 trainable=False)(second_input)
m2 = Bidirectional(LSTM(LSTM_SIZE,dropout=0.2, recurrent_dropout=0.2))(m2)

# MERGE MODEL
merged = Concatenate(axis=1)([m1, m2])
output_layer = Dense(3, activation='softmax')(merged)

model = Model(inputs=[first_input, second_input], outputs=output_layer)
model.compile(optimizer=optimizer, loss=loss,metrics=metrics)
"""

"\n\n# create a weight matrix for words in training docs\nembedding_matrix = np.zeros((vocab_size, EMBED_SIZE))\nfor word, i in t.word_index.items():\n    embedding_vector = embeddings_index.get(word)\n    if embedding_vector is not None:\n        embedding_matrix[i] = embedding_vector\n\n# FIRST MODEL: TITLE1_EN\nfirst_input = Input(shape=(SENTENCE_SIZE,))\nm1 = Embedding(vocab_size,\n                EMBED_SIZE,\n                weights=[embedding_matrix],\n                input_length=SENTENCE_SIZE,\n                trainable=False)(first_input)\nm1 = Bidirectional(LSTM(LSTM_SIZE,dropout=0.2, recurrent_dropout=0.2))(m1)\n\n# SECOND MODEL: TITLE2_EN\nsecond_input = Input(shape=(SENTENCE_SIZE,))\nm2 = Embedding(vocab_size,\n                 EMBED_SIZE,\n                 weights=[embedding_matrix],\n                 input_length=SENTENCE_SIZE,\n                 trainable=False)(second_input)\nm2 = Bidirectional(LSTM(LSTM_SIZE,dropout=0.2, recurrent_dropout=0.2))(m2)\n\n# MERGE MODEL\nme

In [7]:
# ************************ MAIN *********************************
import pandas as pd

input_test = './data/test.csv'

# load test.csv into a dataframe and preprocess it
df_test = pd.read_csv(input_test,encoding='utf-8-sig',error_bad_lines=False)
dataFrame_test = pd.DataFrame(df_test)
dataFrame_test = dataFrame_test.drop(columns=['tid1', 'tid2', 'title1_en', 'title1_zh', 'title2_zh', 'title2_en']) # drop columns

# load model, run prediction for each row, add id and result of prediction to df
model = tf.keras.models.load_model('./models/StackedLSTM-50E-50L-1573843166.6838348.hdf5')

# **** CONCERN: which model to use?? all?? ****

label_list = []
numRows = dataFrame_test.shape[0] # gives number of row count
print(numRows)
print()

for i in range(0, numRows):
    first_sentence = X_train[i][:SENTENCE_SIZE].reshape(1,SENTENCE_SIZE)
    second_sentence = X_train[i][SENTENCE_SIZE:SENTENCE_SIZE*2].reshape(1,SENTENCE_SIZE)

    prediction = model.predict([first_sentence,second_sentence])
    prediction_list = [int(round(p)) for p in prediction[0]]
    
    label = ''
    if prediction_list[0] == 1:
        label = 'unrelated'
    elif prediction_list[1] == 1:
        label = 'agreed'
    elif prediction_list[1] == 1:
        label = 'disagreed'
    label_list.append(label)
    print(i, ' ', label)

"""
# preprocess the csv and tokenize it
new_t = create_tokenizer(dataFrame_test, num_words=40000, oov_token=None)
df_test_encoded = encode_labels(dataFrame_test)
df_test_tokenized = tokenize(t,df_test_encoded,25)

# create a new csv (just writing to one i created)
#prediction = model.predict(...)
prediction = pd.DataFrame(prediction)
"""

# **** CONCERN: corresponding id's with prediction ****

# write dataframe to csv
dataFrame_test.insert(1, 'label', label_list) #replace 0 with prediction data
dataFrame_test.to_csv('./data/submission.csv', index = False)

# ***************************************************************

b'Skipping line 47254: expected 7 fields, saw 9\n'


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
64142

0   unrelated
1   agreed
2   unrelated
3   unrelated
4   unrelated
5   unrelated
6   unrelated
7   agreed
8   unrelated
9   unrelated
10   unrelated
11   unrelated
12   unrelated
13   unrelated
14   agreed
15   agreed
16   unrelated
17   agreed
18   unrelated
19   agreed
20   unrelated
21   unrelated
22   unrelated
23   unrelated
24   unrelated
25   agreed
26   agreed
27   agreed
28   agreed
29   unrelated
30   unrelated
31   agreed
32   agreed
33   unrelated
34   agreed
35   agreed
36   unrelated
37   unrelated
38   unrelated
39   unrelated
40   unrelated
41   unrelated
42   
43   agreed
44   unrelated
45   agreed
46   unrelated
47   agreed
48   unrelated
49   unrelated
50   agreed
51   agreed
52   unrelated
53   unrelated
54   agreed
55   unrelated
56   agreed
57   unrelated
58   unrelated
59   unrelated
60   agreed
61   unrelated
62   unrelated
63   unrelated


527   unrelated
528   unrelated
529   agreed
530   unrelated
531   agreed
532   unrelated
533   unrelated
534   unrelated
535   unrelated
536   unrelated
537   agreed
538   unrelated
539   unrelated
540   unrelated
541   agreed
542   unrelated
543   unrelated
544   agreed
545   agreed
546   unrelated
547   unrelated
548   agreed
549   agreed
550   unrelated
551   unrelated
552   agreed
553   unrelated
554   agreed
555   unrelated
556   unrelated
557   unrelated
558   agreed
559   unrelated
560   unrelated
561   unrelated
562   unrelated
563   unrelated
564   unrelated
565   agreed
566   unrelated
567   unrelated
568   unrelated
569   agreed
570   agreed
571   unrelated
572   agreed
573   unrelated
574   unrelated
575   unrelated
576   unrelated
577   agreed
578   agreed
579   unrelated
580   unrelated
581   unrelated
582   unrelated
583   unrelated
584   unrelated
585   unrelated
586   unrelated
587   unrelated
588   agreed
589   agreed
590   unrelated
591   unrelated
592   unrelated
5

1074   unrelated
1075   unrelated
1076   unrelated
1077   unrelated
1078   unrelated
1079   agreed
1080   unrelated
1081   agreed
1082   unrelated
1083   unrelated
1084   agreed
1085   unrelated
1086   unrelated
1087   unrelated
1088   agreed
1089   unrelated
1090   unrelated
1091   unrelated
1092   unrelated
1093   unrelated
1094   agreed
1095   unrelated
1096   unrelated
1097   unrelated
1098   unrelated
1099   agreed
1100   unrelated
1101   agreed
1102   agreed
1103   unrelated
1104   
1105   unrelated
1106   unrelated
1107   agreed
1108   agreed
1109   agreed
1110   unrelated
1111   unrelated
1112   unrelated
1113   unrelated
1114   agreed
1115   agreed
1116   unrelated
1117   unrelated
1118   unrelated
1119   unrelated
1120   unrelated
1121   unrelated
1122   unrelated
1123   unrelated
1124   unrelated
1125   unrelated
1126   unrelated
1127   unrelated
1128   agreed
1129   unrelated
1130   unrelated
1131   unrelated
1132   agreed
1133   unrelated
1134   unrelated
1135   agreed
113

1593   unrelated
1594   unrelated
1595   unrelated
1596   agreed
1597   unrelated
1598   unrelated
1599   unrelated
1600   unrelated
1601   unrelated
1602   unrelated
1603   unrelated
1604   unrelated
1605   unrelated
1606   unrelated
1607   unrelated
1608   unrelated
1609   agreed
1610   unrelated
1611   unrelated
1612   unrelated
1613   unrelated
1614   agreed
1615   unrelated
1616   unrelated
1617   agreed
1618   unrelated
1619   unrelated
1620   unrelated
1621   unrelated
1622   unrelated
1623   
1624   unrelated
1625   unrelated
1626   agreed
1627   unrelated
1628   unrelated
1629   unrelated
1630   agreed
1631   unrelated
1632   unrelated
1633   unrelated
1634   unrelated
1635   unrelated
1636   unrelated
1637   unrelated
1638   unrelated
1639   agreed
1640   unrelated
1641   unrelated
1642   unrelated
1643   unrelated
1644   agreed
1645   agreed
1646   unrelated
1647   unrelated
1648   agreed
1649   unrelated
1650   agreed
1651   agreed
1652   agreed
1653   unrelated
1654   unre

...

62071   unrelated
62072   unrelated
62073   unrelated
62074   unrelated
62075   unrelated
62076   agreed
62077   unrelated
62078   unrelated
62079   unrelated
62080   unrelated
62081   unrelated
62082   unrelated
62083   unrelated
62084   unrelated
62085   unrelated
62086   unrelated
62087   unrelated
62088   
62089   unrelated
62090   unrelated
62091   unrelated
62092   unrelated
62093   unrelated
62094   unrelated
62095   agreed
62096   unrelated
62097   agreed
62098   unrelated
62099   agreed
62100   agreed
62101   unrelated
62102   unrelated
62103   agreed
62104   unrelated
62105   unrelated
62106   agreed
62107   unrelated
62108   unrelated
62109   agreed
62110   unrelated
62111   unrelated
62112   unrelated
62113   unrelated
62114   agreed
62115   unrelated
62116   agreed
62117   unrelated
62118   unrelated
62119   unrelated
62120   unrelated
62121   unrelated
62122   unrelated
62123   unrelated
62124   unrelated
62125   unrelated
62126   unrelated
62127   unrelated
62128   agree

62551   unrelated
62552   unrelated
62553   unrelated
62554   unrelated
62555   agreed
62556   unrelated
62557   unrelated
62558   unrelated
62559   agreed
62560   unrelated
62561   unrelated
62562   unrelated
62563   unrelated
62564   agreed
62565   unrelated
62566   unrelated
62567   unrelated
62568   unrelated
62569   unrelated
62570   agreed
62571   unrelated
62572   unrelated
62573   unrelated
62574   unrelated
62575   unrelated
62576   agreed
62577   unrelated
62578   unrelated
62579   unrelated
62580   agreed
62581   unrelated
62582   unrelated
62583   agreed
62584   agreed
62585   unrelated
62586   agreed
62587   unrelated
62588   unrelated
62589   unrelated
62590   unrelated
62591   agreed
62592   unrelated
62593   unrelated
62594   unrelated
62595   unrelated
62596   unrelated
62597   unrelated
62598   unrelated
62599   unrelated
62600   unrelated
62601   unrelated
62602   unrelated
62603   agreed
62604   unrelated
62605   agreed
62606   unrelated
62607   unrelated
62608   un

63030   unrelated
63031   unrelated
63032   unrelated
63033   unrelated
63034   agreed
63035   unrelated
63036   unrelated
63037   agreed
63038   agreed
63039   unrelated
63040   agreed
63041   unrelated
63042   unrelated
63043   unrelated
63044   unrelated
63045   unrelated
63046   unrelated
63047   unrelated
63048   unrelated
63049   unrelated
63050   unrelated
63051   unrelated
63052   unrelated
63053   
63054   unrelated
63055   agreed
63056   unrelated
63057   unrelated
63058   agreed
63059   agreed
63060   unrelated
63061   unrelated
63062   unrelated
63063   unrelated
63064   unrelated
63065   unrelated
63066   unrelated
63067   unrelated
63068   unrelated
63069   unrelated
63070   unrelated
63071   unrelated
63072   unrelated
63073   unrelated
63074   unrelated
63075   unrelated
63076   agreed
63077   unrelated
63078   agreed
63079   unrelated
63080   unrelated
63081   
63082   unrelated
63083   agreed
63084   unrelated
63085   agreed
63086   agreed
63087   unrelated
63088   un

63519   unrelated
63520   unrelated
63521   unrelated
63522   unrelated
63523   unrelated
63524   unrelated
63525   unrelated
63526   unrelated
63527   unrelated
63528   unrelated
63529   unrelated
63530   unrelated
63531   unrelated
63532   unrelated
63533   unrelated
63534   unrelated
63535   agreed
63536   unrelated
63537   unrelated
63538   unrelated
63539   unrelated
63540   unrelated
63541   unrelated
63542   unrelated
63543   agreed
63544   unrelated
63545   unrelated
63546   unrelated
63547   unrelated
63548   agreed
63549   agreed
63550   unrelated
63551   agreed
63552   unrelated
63553   unrelated
63554   unrelated
63555   unrelated
63556   unrelated
63557   unrelated
63558   unrelated
63559   agreed
63560   agreed
63561   agreed
63562   unrelated
63563   agreed
63564   unrelated
63565   unrelated
63566   unrelated
63567   unrelated
63568   unrelated
63569   unrelated
63570   unrelated
63571   unrelated
63572   agreed
63573   unrelated
63574   unrelated
63575   unrelated
6357

64014   unrelated
64015   agreed
64016   agreed
64017   agreed
64018   unrelated
64019   unrelated
64020   unrelated
64021   unrelated
64022   unrelated
64023   unrelated
64024   agreed
64025   unrelated
64026   unrelated
64027   agreed
64028   agreed
64029   agreed
64030   agreed
64031   unrelated
64032   unrelated
64033   unrelated
64034   agreed
64035   agreed
64036   unrelated
64037   unrelated
64038   agreed
64039   unrelated
64040   unrelated
64041   unrelated
64042   agreed
64043   unrelated
64044   unrelated
64045   agreed
64046   agreed
64047   unrelated
64048   agreed
64049   unrelated
64050   unrelated
64051   unrelated
64052   unrelated
64053   agreed
64054   unrelated
64055   unrelated
64056   unrelated
64057   unrelated
64058   unrelated
64059   unrelated
64060   unrelated
64061   unrelated
64062   unrelated
64063   agreed
64064   unrelated
64065   unrelated
64066   unrelated
64067   unrelated
64068   agreed
64069   unrelated
64070   unrelated
64071   unrelated
64072   ag